In [1]:
import numpy as np
import numba as nb

import ice_funcs as icef

## Todo:
- ***Make sure that diffusion works!!***
- ***Figure out if I actually need to hold on to timings... and if yes... how ?***
    - Is the timing array even needed &&&**[do not delete, just comment out]**&&&
- Make relaxSaturationUtilities fully OOP


## Backburner-type-beat:
- make sure everything is numbafied
- Continue commenting in `ice_funcs.py`

## Improvements
- Monopole matching
- correct geometry factors $G_b$ and $H_b$
- Causal timings?

In [2]:
# CONSTANTS

L = 101
W = (L+1)//2
INITIAL_SAT = 1

In [3]:
gu = icef.GeneralUtilities(101) # general utilities

pu = icef.PhysicsUtilities(0.01, 10, X_0=2) # physics utilities



In [4]:
sru = icef.SaturationRelaxationUtilities(101, 200)

In [5]:
neighbor_array_test = gu.neighbor_array
diffusion_rules_test = sru.diffusion_rules

print(diffusion_rules_test)

[[[-2.6711958e+07 -2.6711958e+07 -2.6711958e+07 -2.6711958e+07
   -2.6711958e+07 -2.6711958e+07]
  [-2.6711958e+07 -2.6711958e+07 -2.6711958e+07 -2.6711958e+07
   -2.6711958e+07 -2.6711958e+07]
  [-2.6711958e+07 -2.6711958e+07 -2.6711958e+07 -2.6711958e+07
   -2.6711958e+07 -2.6711958e+07]
  ...
  [ 5.6051939e-45  9.4039548e-38  0.0000000e+00  0.0000000e+00
    0.0000000e+00  0.0000000e+00]
  [ 1.9259299e-33  6.8663625e-44 -3.5486770e+31  3.0682831e-41
   -3.5487389e+31  3.0682831e-41]
  [-3.5485532e+31  3.0682831e-41  1.1093356e-31  6.8663625e-44
   -3.5486770e+31  3.0682831e-41]]

 [[ 1.6666667e-01  3.3333334e-01  3.3333334e-01  1.6666667e-01
    0.0000000e+00  0.0000000e+00]
  [ 1.6666667e-01  1.6666667e-01  1.6666667e-01  1.6666667e-01
    1.6666667e-01  1.6666667e-01]
  [ 1.6666667e-01  1.6666667e-01  1.6666667e-01  1.6666667e-01
    1.6666667e-01  1.6666667e-01]
  ...
  [ 1.6666667e-01  1.6666667e-01  1.6666667e-01  1.6666667e-01
    1.6666667e-01  1.6666667e-01]
  [ 1.6666667e-0

In [6]:
default_ice_map = np.full((101, 101), False)
default_ice_map[-1,0] = True
default_ice_map[-2,0] = True
default_ice_map[-3,0] = True
default_ice_map[-3,1] = True

# %timeit test_boundary_map
boundary_map_test = gu.construct_boundary_map(default_ice_map)
print(boundary_map_test[-5:,:5])

sat_map_test = np.full((101,101), 1, dtype=np.float32)

normal_cells_test, boundary_cells_test = gu._distinguish_cells(default_ice_map, boundary_map_test)

opp_array_test = sru._construct_opp_array(boundary_cells_test, default_ice_map, neighbor_array_test)

[[0 0 0 0 0]
 [1 2 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]


In [10]:
# pu.apply_boundary_condition(2, 2, sat_map_test, 0.5, boundary_map_test)

sat_map_test = np.full((101,101), 1, dtype=np.float32)

# for i in range(50):
#     sru._execute_relaxation_step(sat_map_test, normal_cells_test, boundary_cells_test, opp_array_test, neighbor_array_test, boundary_map_test)

new_thang = sru.diffuse_to_convergence(sat_map_test, 200, 0.00001, normal_cells_test, boundary_cells_test, default_ice_map, boundary_map_test, neighbor_array_test, pu)
print(new_thang[1])
print(new_thang[2])
print(new_thang[0][-10:,:5])

False
200
[[0.9937453  0.9931462  0.99264514 0.9922927  0.99213755]
 [0.9923508  0.9916322  0.9910597  0.99070376 0.99058825]
 [0.9906505  0.9897847  0.9891409  0.9888431  1.        ]
 [0.9885575  0.9875037  0.98678505 0.986533   1.        ]
 [0.9859507  0.98463356 0.9838852  1.         1.        ]
 [0.9827002  0.980803   0.9797492  1.         1.        ]
 [0.97920275 0.9744718  1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]
 [1.         1.         1.         1.         1.        ]]
